#### Secure Config Setup 

In [0]:
bootstrap = dbutils.secrets.get('confluent-secrets', 'kafka.bootstrap')
k_user   = dbutils.secrets.get('confluent-secrets', 'kafka.username')
k_pass   = dbutils.secrets.get('confluent-secrets', 'kafka.password')
sr_url   = dbutils.secrets.get('confluent-secrets', 'sr.url')
sr_key   = dbutils.secrets.get('confluent-secrets', 'sr.key')
sr_sec   = dbutils.secrets.get('confluent-secrets', 'sr.secret')

topic    = 'orders.v1'
consumer_group = 'orders-consumer-dbx'

kafka_opts = {
  'kafka.bootstrap.servers': bootstrap,
  'subscribe': topic,
  'startingOffsets': 'latest',  # 'earliest' for backfills
  'failOnDataLoss': 'false',
  'kafka.security.protocol': 'SASL_SSL',
  'kafka.sasl.mechanism': 'PLAIN',
  'kafka.sasl.jaas.config': f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{k_user}" password="{k_pass}";'
}

schema_subject = f'{topic}-value'  # Confluent default subject naming

#### Fetch latest Avro schema from Schema Registry

In [0]:
import requests
from requests.auth import HTTPBasicAuth

resp = requests.get(f'{sr_url}/subjects/{schema_subject}/versions/latest',
                    auth=HTTPBasicAuth(sr_key, sr_sec))
resp.raise_for_status()
writer_schema = resp.json()['schema']  # JSON string